###  OPEN DART API 활용

In [29]:
from pprint import pprint
import os
import zipfile
import io
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import json
from lxml import etree

In [ ]:
# API KEY 불러오기
with open('./api_keys.json', 'r') as file:
    data = json.load(file)
dart_key = data[0].get('opendart_api_key')

In [28]:
dart_key

'229e698290d7d6fbfca8cfa08a860a1d339cae25'

In [9]:
# CORPCODE.XML 다운로드
PATH = f"https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={dart_key}"
response = requests.get(PATH)
if response.status_code == 200:
    zipfile.ZipFile(io.BytesIO(response.content)).extractall('./')


In [22]:
# CORPCODE.xml을 기반으로 한 pd.DataFrame 만들기
tree = ET.parse('./CORPCODE.xml')
root = tree.getroot()
corp_items = []
for corp_data in root.iter('list'):
    item = {
        "corp_code": (corp_data.findtext('corp_code') or '').strip(),
        "corp_name": (corp_data.findtext('corp_name') or '').strip(),
        "corp_eng_name": (corp_data.findtext('corp_eng_name') or '').strip(),
        "stock_code": (corp_data.findtext('stock_code') or '').strip(),
        "modify_date": (corp_data.findtext('modify_date') or '').strip(),
    }
    corp_items.append(item)

df = pd.DataFrame(corp_items)
df

,corp_code,corp_name,corp_eng_name,stock_code,modify_date
0,00434003,다코,Daco corporation,,20170630
1,00430964,굿앤엘에스,"Good & LS Co.,Ltd.",,20170630
2,00388953,크레디피아제이십오차유동화전문회사,Credipia 25th Asset Securitization Specialty L...,,20170630
3,00179984,연방건설산업,youn bao,,20170630
4,00420143,브룩스피알아이오토메이션잉크,"BROOKS-PRI Automation, Inc.",,20170630
...,...,...,...,...,...
113083,01913987,지엠에프십사호선박투자회사,GMF NO.14 Ship Investment Company,,20250423
113084,01913996,지엠에프십오호선박투자회사,GMF NO.15 Ship Investment Company,,20250423
113085,01914001,지엠에프십육호선박투자회사,GMF NO.16 Ship Investment Company,,20250423
113086,01913127,디지탈테크,"digitalltek co.,ltd.",,20250423


### 활용할 API 내 보고서 양식 목록
1. 정기 보고서
2. 분기 보고서


In [24]:
BASE_URL = "https://opendart.fss.or.kr/api"

In [25]:
# 공시서류원본파일 확인용
url = BASE_URL+'/document.xml'
payload = {'crtfc_key': dart_key, 'rcept_no':20190401004781}
response = requests.get(url, params=payload)
response.status_code

200

In [27]:
zipfile.ZipFile(io.BytesIO(response.content)).extractall('./corp_reports/')

In [39]:
def fetch_documents(api_key, rcp_no):
    url = BASE_URL+'/document.xml'
    params = {'crtfc_key': api_key, 'rcept_no':rcp_no}
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print('Bad Response')
        raise Exception(f'API 요청 실패: 상태코드 {response.status_code}')
    else:
        return response.content
    
def extract_report(zip_content, rcp_no):
    try:
        with zipfile.ZipFile(io.BytesIO(zip_content)) as zf:
            audit_fnames = [info.filename for info in zf.infolist() if f"{rcp_no}_00760.xml" in info.filename]
            if not audit_fnames:
                raise ValueError("감사보고서 파일을 찾을 수 없습니다.")
            xml_data = zf.read(audit_fnames[0])
            return xml_data
    except  zipfile.BadZipFile:
        raise ValueError("ZIP파일이 손상되었거나 유효하지 않습니다.")
    
def parse_html_from_xml(xml_data):
    try:
        xml_text = xml_data.decode('euc-kr')
    except UnicodeDecodeError:
        xml_text = xml_data.decode('utf-8')
    parser = etree.HTMLParser()
    root = etree.fromstring(xml_text.encode('utf-8'), parser)
    return root

api_key = dart_key
rcp_no = '20190401004781'

zip_content = fetch_documents(api_key, rcp_no)
xml_data = extract_report(zip_content, rcp_no)
root = parse_html_from_xml(xml_data)


In [40]:
root.text

### 사업보고서 주요정보 추출

In [ ]:
# 

AttributeError: 'tuple' object has no attribute 'decode'